In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

import os
from tqdm.notebook import tqdm
import SimpleITK as sitk

In [ ]:
TRAIN_DIR = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/'
OUT_FOLDER = 'train'
MRI_TYPES = ['T1w', 'T1wCE','T2w', 'FLAIR']
EXT = 'jpg'

DEBUG = False

In [ ]:
reader = sitk.ImageSeriesReader()
reader.LoadPrivateTagsOn()

In [ ]:
train_labels = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')

In [ ]:
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
# dicom = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-1.dcm')

In [ ]:
def get_meta_info_v2(dicom):
    ret_dict = {dicom.get(k).name.replace(' ',''): dicom.get(k).value for k in dicom.keys() if dicom.get(k).name != 'Pixel Data'}
    ret_dict['timestamp'] = dicom.timestamp
    return ret_dict

In [ ]:
# meta = get_meta_info_v2(dicom)
# meta2 = meta.copy()
# meta2['abc'] = 111

In [ ]:
# pd.DataFrame([meta, meta2])

In [ ]:
def process_dicom(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.dcmread(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    # read other meta
    meta = get_meta_info_v2(dicom)
    
    return data, meta

In [ ]:
def resample(image, ref_image):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ref_image)
    resampler.SetInterpolator(sitk.sitkLinear)
    
    resampler.SetTransform(sitk.AffineTransform(image.GetDimension()))

    resampler.SetOutputSpacing(ref_image.GetSpacing())

    resampler.SetSize(ref_image.GetSize())

    resampler.SetOutputDirection(ref_image.GetDirection())

    resampler.SetOutputOrigin(ref_image.GetOrigin())

    resampler.SetDefaultPixelValue(image.GetPixelIDValue())

    resamped_image = resampler.Execute(image)
    
    return resamped_image

def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
train_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/'
patient_id = 688
series_path = f'{train_path}/{patient_id:5d}/T1w'
print(series_path)
filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{patient_id:05d}/T1wCE')
reader.SetFileNames(filenamesDICOM)
ref_voxels = reader.Execute()

In [ ]:
writer = sitk.ImageFileWriter()

In [ ]:
patient_ids = []
image_names = []
mri_types = []
metas = []

mri_type_mapping = {
    'T1w':'t1',
    'T1wCE':'t1ce',
    'T2w':'t2',
    'FLAIR':'flair'
}

iterations = tqdm(os.listdir(TRAIN_DIR)) if not DEBUG else tqdm(os.listdir(TRAIN_DIR)[:3])

for patient_id in iterations:
    patient_dir = os.path.join(TRAIN_DIR, patient_id) 
    for mri_type in MRI_TYPES:
        type_dir = os.path.join(patient_dir, mri_type)
        out_dir = os.path.join(OUT_FOLDER, patient_id, mri_type)
#         os.makedirs(out_dir, exist_ok=True)
            # read dicom series at once
        try:
            filenamesDICOM = reader.GetGDCMSeriesFileNames(type_dir)
            reader.SetFileNames(filenamesDICOM)
            voxels = reader.Execute()
            resampled_voxels = resample(voxels, ref_voxels)
#             resampled_voxels_arr = sitk.GetArrayViewFromImage(resampled_voxels)
#             transposed_resampled_voxels_arr = resampled_voxels_arr.T
#             processed_voxels = sitk.GetImageFromArray(transposed_resampled_voxels_arr)
            
            processed_voxels = resampled_voxels
            outputImageFileName = os.path.join('BraTS2021_Training_Data', f'BraTS2021_{patient_id}', 
                                               f'BraTS2021_{patient_id}_{mri_type_mapping[mri_type]}.nii.gz')
            os.makedirs(os.path.dirname(outputImageFileName), exist_ok=True)
            writer.SetFileName(outputImageFileName)
            writer.Execute(resampled_voxels)

        except Exception as ex:
            print(ex)
#         break
#     break

In [ ]:
outputImageFileName

In [ ]:
ls BraTS2021_Training_Data/BraTS2021_00688/

In [ ]:
import matplotlib.pyplot as plt

def visualize_3_planes(voxels):
    plt.figure(figsize=(9,3))
    plt.subplot(1,3,1)
    plt.imshow(voxels[voxels.shape[0]//2])
    plt.subplot(1,3,2)
    plt.imshow(voxels[:, voxels.shape[1]//2, :])
    plt.subplot(1,3,3)
    plt.imshow(voxels[:,:,voxels.shape[2]//2])

In [ ]:
final_arr = sitk.GetArrayViewFromImage(processed_voxels)

In [ ]:
final_arr.shape

In [ ]:
visualize_3_planes(final_arr)

In [ ]:
# visualize_3_planes(resampled_voxels_arr)

In [ ]:
!zip -r BraTS2021_Training_Data.zip BraTS2021_Training_Data/ >> log.txt

In [ ]:
!rm -rf BraTS2021_Training_Data

In [ ]:
# # notes:
# - Align voxel with a ref voxel
# - Visualize to compare with data from task 1 (2 plane plots)